In [ ]:
import tensorflow as tf 
import numpy as np 

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
train_images  = train_images.reshape(-1, 28, 28, 1).astype("float32")/255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype('float32')/255.0

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)), 
    tf.keras.layers.MaxPooling2D((2, 2)), 
    tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu'), 
    tf.keras.layers.MaxPooling2D((2, 2)), 
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax')

])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=2, validation_split=0.1)


In [6]:
#keras
model.save('mnist_saved_keras.keras')

In [ ]:
#h5
model.save('mnist_saved_h5.h5')

In [ ]:
#tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('mnist_saved_tflite.tflite', "wb") as f :
    f.write(tflite_model)



In [ ]:
#ONNX
import tf2onnx
import onnx

model.output_names = ['output']


spec =(tf.TensorSpec((None, 28, 28, 1), tf.float32, name='input'), )
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec)
onnx.save_model(onnx_model, 'mnist_saved_onnx.onnx')

In [10]:
#size of output models
import os


def get_file_size(file_name, file_path):
    file_size = os.path.getsize(file_path)
    file_size = file_size / (1024 * 1024)
    print(f"{file_name} size: {file_size:.2f} MB")

In [ ]:
get_file_size('h5 format', 'mnist_saved_h5.h5')
get_file_size('keras format', 'mnist_saved_keras.keras')
get_file_size('onnx format', 'mnist_saved_onnx.onnx')
get_file_size('tflite format', 'mnist_saved_tflite.tflite')

In [21]:
#inference_time : h5
import time
from PIL import Image


model = tf.keras.models.load_model('mnist_saved_h5.h5')
#model.summary()


def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')
    img = img.resize((28, 28))

    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0

    img_array = img_array.reshape(1, 28, 28, 1)

    return img_array

image_path = 'three.jpg'

start_time = time.time()

test_image = preprocess_image(image_path)

predictions = model.predict(test_image)
predicted_class = np.argmax(predictions, axis=1)

end_time = time.time()

h5_inference_time = end_time - start_time

print( 'predictied_class: ', predicted_class[0])
print("h5_inference_time", h5_inference_time)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
predictied_class:  3
h5_inference_time 0.0717313289642334


In [25]:
#tflite inference time

tflite_model_path = 'mnist_saved_tflite.tflite'


interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


image_path = 'three.jpg'


test_image = preprocess_image(image_path)

start_time = time.time()

interpreter.set_tensor(input_details[0]['index'], test_image)

interpreter.invoke()

predictions = interpreter.get_tensor(output_details[0]['index'])

end_time = time.time()

predicted_class = np.argmax(predictions, axis=1)
tflite_inference_time = end_time - start_time

print( 'predictied_class: ', predicted_class[0])
print("tflite_inference_time", tflite_inference_time)



predictied_class:  3
tflite_inference_time 0.0005831718444824219


In [36]:
import onnx
import onnxruntime as ort

onnx_model_path = 'mnist_saved_onnx.onnx'
onnx_model = onnx.load(onnx_model_path)


#input_shape = onnx_model.graph.input[0].type.tensor_type.shape.dim
#print(input_shape)


session = ort.InferenceSession(onnx_model_path)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


img = 'three.jpg'

start_time = time.time()

test_image = preprocess_image(img)

predictions = session.run([output_name], {input_name : test_image})

end_time = time.time()
predicted_class = np.argmax(predictions[0], axis=1)

onnx_inference_time = end_time - start_time

print( 'predictied_class: ', predicted_class[0])
print("onnx_inference_time", onnx_inference_time)







predictied_class:  3
onnx_inference_time 0.0019366741180419922


In [38]:
print("h5_inference_time", h5_inference_time, 'seconds')
print("onnx_inference_time", onnx_inference_time, 'seconds')
print("tflite_inference_time", tflite_inference_time , 'seconds')

h5_inference_time 0.0717313289642334 seconds
onnx_inference_time 0.0019366741180419922 seconds
tflite_inference_time 0.0005831718444824219 seconds


In [ ]:
#Accuracy